*Author @ T. Gao, revision: may 5, 2020*

About this project: 
===============
Simulation of a basic RPG (Role Play Game) and its optimal level-up strategy via **heuristic, rollout, forward recursion and ADP**. 

                            /|
                          /'||
                         |  ||
                         |  ||
                         |  ||
                         |  ||
                         |  ||
                         |  ||
                         |  ||
                         |  ||
                         |  ||
                         |  ||
                         |  ||
                         |  ||
                         |  ||
                         |  ||
                         |  ||         __.--._
                         |  ||      /~~   __.-~\ _
                         |  ||  _.-~ / _---._ ~-\/~\
                         |  || // /  /~/  .-  \  /~-\
                         |  ||((( /(/_(.-(-~~~~~-)_/ |
                         |  || ) (( |_.----~~~~~-._\ /
                         |  ||    ) |              \_|
                         |  ||     (| =-_   _.-=-  |~)        ,
                         |  ||      | `~~ |   ~~'  |/~-._-'/'/_,
                         |  ||       \    |        /~-.__---~ , ,
                         |  ||       |   ~-''     || `\_~~~----~
                         |  ||_.ssSS$$\ -====-   / )\_  ~~--~
                 ___.----|~~~|%$$$$$$/ \_    _.-~ /' )$s._
        __---~-~~        |   |%%$$$$/ /  ~~~~   /'  /$$$$$$$s__
      /~       ~\    ============$$/ /        /'  /$$$$$$$$$$$SS-.
    /'      ./\\\\\\_( ~---._(_))$/ /       /'  /$$$$%$$$$$~      \
    (      //////////(~-(..___)/$/ /      /'  /$$%$$%$$$$'         \
     \    |||||||||||(~-(..___)$/ /  /  /'  /$$$%$$$%$$$            |
      `-__ \\\\\\\\\\\(-.(_____) /  / /'  /$$$$%$$$$$%$             |
          ~~""""""""""-\.(____) /   /'  /$$$$$%%$$$$$$\_            /
                        $|===|||  /'  /$$$$$$$%%%$$$$$( ~         ,'|
                    __  $|===|%\/'  /$$$$$$$$$$$%%%%$$|        ,''  |
                   ///\ $|===|/'  /$$$$$$%$$$$$$$%%%%$(            /'
                    \///\|===|  /$$$$$$$$$%%$$$$$$%%%%$\_-._       |
                     `\//|===| /$$$$$$$$$$$%%%$$$$$$-~~~    ~      /
                       `\|-~~(~~-`$$$$$$$$$%%%///////._       ._  |
                       (__--~(     ~\\\\\\\\\\\\\\\\\\\\        \ \
                       (__--~~(       \\\\\\\\\\\\\\\\\\|        \/
                        (__--~(       ||||||||||||||||||/       _/
                         (__.--._____//////////////////__..---~~
                         |   """"'''''           ___,,,,ss$$$%
                        ,%\__      __,,,\sssSS$$$$$$$$$$$$$$%%
                      ,%%%%$$$$$$$$$$\;;;;\$$$$$$$$$$$$$$$$%%%$.
                     ,%%%%%%$$$$$$$$$$%\;;;;\$$$$$$$$$$$$%%%$$$$
                   ,%%%%%%%%$$$$$$$$$%$$$\;;;;\$$$$$$$$$%%$$$$$$,
                  ,%%%%%%%%%$$$$$$$$%$$$$$$\;;;;\$$$$$$%%$$$$$$$$
                 ,%%%%%%%%%%%$$$$$$%$$$$$$$$$\;;;;\$$$%$$$$$$$$$$$
                 %%%%%%%%%%%%$$$$$$$$$$$$$$$$$$\;;;$$$$$$$$$$$$$$$
                   ""==%%%%%%%$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$"
                               $$$$$$$$$$$$$$$$$$$$====""""
                                 """""""""~~~~
 
ASCII Art Credit: Tua Xiong

Requirements:
1. **Python 3+ environment**
2. **Numpy library**
3. **A multi-core CPU (for multiprocessing)**

In [1]:
import numpy as np
import multiprocessing
from copy import deepcopy
# set seed for reproducibility
# np.random.seed(42) 

Create the game world!
===================

**before we start, consider building a simple database for saving computed boss win rate at a certain stage to avoid repeated computation** 

This could be beneficial in certain scenarios. However, using a shared dict within multiple processes might be tricky and very slow (I/O bottleneck)

In [2]:
class DB_Win_Rate:
    
    def __init__(self):
        self.db = dict()
        
        
    # helper functions
    # hero's win rate a certain level of boss
    # is solely influenced by hero's current hp, attack power and defense power
    @staticmethod
    def encode_db_key(cur_hp, att, def_,):
        return f"{cur_hp}/{att}/{def_}"
    
    
    @staticmethod
    def decode_db_key(db_key):
        cur_hp, att, def_, = db_key.split('/')
        return cur_hp, att, def_,
    
    
    def save_win_rate(self, db_key, lv_boss, win_rate, avg_hp):
        # prevents frequent dict update
        if (db_key, lv_boss) not in self.db:
            self.db[(db_key, lv_boss)] = (win_rate, avg_hp)
        
        
    def get_win_rate(self, db_key, lv_boss):
        if (db_key, lv_boss) in self.db:
            return self.db[(db_key, lv_boss)]
        else:
            return None
        
    def update(self, new_dict):
        self.db.update(new_dict)

        
# instantiate a new DB Win Rate database for future computation
db_win_rate = DB_Win_Rate()

Now create a basic warrior unit

In [3]:
# Declare the Warrior class
# All Viking warriors have three basic attributes:
# @ HP/VIT: max_vitality
# @ ATT: attack_power
# @ DEF: defense_power


# Creates a basic Warrior unit in the game world
# Stores the warrior's basic attributes 
# Enables the warrior to restore health and attack
class Warrior:
    def __init__ (self, warrior_name):
        self.name = warrior_name  
        self.max_vitality = 50  # MAX_HP (health points) of a warrior (default to 50)
        self.current_vitality = 50
        self.attack_power = 1   # ATT
        self.attack_modifier = (0.8, 1.2)  # applies an effect in a range to warrior's attack power
                                            # i.e. the actual damage = att_modifier * att_power
        self.defense_power = 1  # DEF
        
    
    # all warriors have the ability to replenish health..
    # there are two way of increasing health
    # 1. points: raise current health by points greater than zero
    # 2. full: restore health to the maximum health level
    # 3. restore HP to half of the max HP
    def restore_health(self, mode='full', value=None):
        if mode == 'points':
            assert value > 0, "Health increment points cannot be negative."
            self.current_vitality = min(self.max_vitality, (self.current_vitality + value))
        elif mode == 'full':  
            self.current_vitality = self.max_vitality
        elif mode == 'half':
            self.current_vitality = int(0.5 * self.max_vitality)
        else:
            raise Exception("Unacceptable Mode.")
           
    
    # this function calculates the attack damage of a warrior PER attack..
    # Input: B, a warrior/hero/boss class object
    # Output: the effective damage of a boss/player attack
    # formula: att_mod (attack_modifier) x A.att_power x (def_mod / (def_mod + B.def_power))
    def attack(self, B):
        # modifies A's attack power (float)
        att_mod = np.random.uniform(low=self.attack_modifier[0], high=self.attack_modifier[1]) 
        
        # tweakable variable
        def_mod = 10 # modifies B's defense power (int)
        
        # estimated damage
        damage = round(att_mod * self.attack_power * (def_mod / (def_mod + B.defense_power)))
        
        return damage

Create our hero based on the warrior class

In [4]:
# A subclass of the warrior class
# This class instantiates the player's character
class Hero(Warrior):
    def __init__ (self, hero_name):
        super().__init__(hero_name)
        
        # Level 1 hero attributes
        self.level = 1  # start from level 1
        self.max_vitality = 100
        self.current_vitality = 100
        self.attack_power = 20
        self.defense_power = 10
    
    
    # Available hero "actions"
    def level_up(self, lv=1):
        self.level += lv
        
        # the following attributes are tweakable
        self.increase_max_vitality(points=int(lv*10))
        self.increase_attack_power(points=lv)
        self.increase_defense_power(points=lv)
        

    def increase_max_vitality(self, points=10):
        full_health = (self.max_vitality == self.current_vitality)
        self.max_vitality += points
        # if hero is in full health, also increases current HP
        if full_health: self.current_vitality = self.max_vitality
            
            
    def increase_attack_power(self, points=1):
        self.attack_power += points
        
    
    def increase_defense_power(self, points=1):
        self.defense_power += points
    
    
    # key generator for accessing the win rate data
    def get_db_key(self):
        cur_hp = self.current_vitality
        att = self.attack_power
        def_ = self.defense_power
        
        db_key = db_win_rate.encode_db_key(cur_hp, att, def_)
        return db_key
    
    
    # performs a trial to determine the chance of winning against a certain level boss
    # input: the number of trials, the boss level 
    # output: the win rate (float)
    def trial(self, n_trials, boss_level, manager_dict=None):
        
        def MCSimulation():
            trial_game = Game(self)
            current_hp = self.current_vitality

            total_wins = 0
            avg_hp = 0  # hero's average hp after the match

            for _ in range(n_trials):
                self.current_vitality = current_hp
                winner = trial_game.match('trial_boss', boss_level)
                if winner == 'hero': 
                    total_wins += 1
                    avg_hp += self.current_vitality

            self.current_vitality = current_hp
            win_rate = total_wins/n_trials
            # avg_hp: average HP left after a match
            avg_hp = 1 if total_wins == 0 else max(1, int(round(avg_hp/total_wins, -1)))
            return (win_rate, avg_hp)
        
        # check if using a multiprocessing manager
        if manager_dict is not None: 
            db_key = self.get_db_key() # compose key to access db
            if (db_key, boss_level) not in manager_dict:
                # simulate by MC and save to manager dict
                manager_dict[(db_key, boss_level)] = MCSimulation()
            return manager_dict[(db_key, boss_level)]
            
        # no read/write state dict required
        # return simulation results directly
        return MCSimulation()      
    
    
    # this is a proxy function for self.trial that runs three simulations at once
    def get_win_rates(self, n_trials, win_rates=None):
        
        manager = multiprocessing.Manager()
        
        if win_rates is not None: # use win rate table
            manager_dict = manager.dict(win_rates)
        else:
            manager_dict = manager.dict()
            
        args = [(n_trials, "mini", manager_dict), 
                (n_trials, "champion", manager_dict), 
                (n_trials, "ultimate", manager_dict)]
        jobs= []
        
        for i in range(3):
            p = multiprocessing.Process(target=self.trial, args=args[i])
            jobs.append(p)
            p.start()
        
        for proc in jobs:
            proc.join()
        
        db_key = self.get_db_key()
        mini_boss_win_rate, _ = manager_dict[(db_key, "mini")]
        champion_win_rate, _ = manager_dict[(db_key, "champion")]
        ult_boss_win_rate, _ = manager_dict[(db_key, "ultimate")]
        
        # update the original win_rates dictionary
        if win_rates is not None: win_rates.update(manager_dict)
        
        return  mini_boss_win_rate, champion_win_rate, ult_boss_win_rate
        
    
    # reset hero stats
    def reborn(self):
        self.__init__(self.name)
        
    
    # modify all hero stats at once
    def any_stats(self, lv, max_hp, cur_hp, att, def_):
        self.level = lv 
        self.max_vitality = max_hp
        self.current_vitality = cur_hp
        self.attack_power = att
        self.defense_power = def_

Create a boss based on the warrior class

In [5]:
# Instatiating a boss given its level
class Boss(Warrior):
    # level 1: mini boss
    # level 2: champion (boss)
    # level 3: ultimate boss
    def __init__ (self, boss_name, boss_level):
        super().__init__(boss_name)
        self.boss_level = boss_level
        self.perk = None  # saves boss's special ability if available
        
        # tweakable features 
        health_range = {'mini': (100, 200), 
                        'champion': (200, 300),
                        'ultimate': (300, 400)}
        
        attack_range = {'mini': (15, 25), 
                        'champion': (25, 50),
                        'ultimate': (50, 70)}
        
        defense_range = {'mini': (10, 20), 
                        'champion': (20, 30),
                        'ultimate': (30, 40)}
        
        # Initalize vitality (round to 10th)
        h_low = health_range[self.boss_level][0]
        h_high = health_range[self.boss_level][1]
        self.max_vitality = round(np.random.randint(low=h_low, high=h_high), -1)
        self.current_vitality = self.max_vitality
        
        # Initalize attack power
        att_low = attack_range[self.boss_level][0]
        att_high = attack_range[self.boss_level][1]
        self.attack_power = np.random.randint(low=att_low, high=att_high)
        
        # Initialize defense power
        def_low = defense_range[self.boss_level][0]
        def_high = defense_range[self.boss_level][1]
        self.defense_power = np.random.randint(low=def_low, high=def_high)
        
        # Initialize boss perk
        perks = ['rage_attack', 'dodging_attack', 'self_healing']
        self.perk = np.random.choice(perks) if self.boss_level == 'ultimate' else None
        
        self.rage_prob = [0.8, 0.2] if self.perk == 'rage_attack' else None
        self.dodge_prob = [0.7, 0.3] if self.perk == 'dodging_attack' else None  
        self.heals_per_round = 5 if self.perk == 'self_healing' else None
        
        
    # boss perk action
    def self_healing(self):
        assert self.heals_per_round is not None, "Boss cannot perform self healing."
        self.restore_health(mode='points', value=self.heals_per_round)

Create the game supporting the match mechanics

In [6]:
# This class instantiates a player playable game
# and calculates the optimal decision sequence per strategy
class Game:
    def __init__ (self, hero):
        self.hero = hero
        self.max_match_round = 50
    
    
    # Simulates a game match between the hero and a boss given the boss level
    # Input: boss name and boss level
    # Output: winner of the simulated match (str: 'boss' or 'hero')
    def match(self, boss_name, boss_level, verbose=False):
        boss = Boss(boss_name, boss_level)  # instantiates a boss
        # simulate the first player (hero or boss) to attack
        upper_hand = np.random.choice(['hero', 'boss'])
        if verbose: 
            if boss_level == 'mini':
                print(f"{boss_name}, boss type: mini boss")
            elif boss_level == 'champion':
                print(f"{boss_name}, boss type: champion")
            else:
                print(f"{boss_name}, boss type: ultimate")
            print(f"Upper hand: {upper_hand}")
        
        winner = None
        
        
        # hero attack procedure
        def hero_attack():
            if boss.perk == 'dodging_attack':
                dodge = np.random.choice(['fail', 'success'], p=boss.dodge_prob)
                if dodge == 'fail':
                    damage = self.hero.attack(boss)
                    if verbose: print(f"{boss.name} dodge fails. {self.hero.name} damage: {damage}.")
                    boss.current_vitality -= damage
                else:
                    if verbose: print(f"{boss.name} dodge succeeds. {self.hero.name} damage: 0.")
            else:
                damage = self.hero.attack(boss)
                if verbose: print(f"{self.hero.name} does damage: {damage}.")
                boss.current_vitality -= damage

                
        # boss attack procedure
        def boss_attack():
            if boss.perk == 'rage_attack':
                double = np.random.choice(['fail', 'success'], p=boss.rage_prob)
                multiplier = 2 if double == 'success' else 1

                damage = multiplier * boss.attack(self.hero)

                if verbose:
                    if double == 'success':
                        print(f"{boss.name} rage attacks. Damage: {damage}")
                    else:
                        print(f"{boss.name} regular attacks. Damage: {damage}")
                        
                self.hero.current_vitality -= damage
            else:
                damage = boss.attack(self.hero)
                if verbose: print(f"{boss.name} does amage: {damage}")
                self.hero.current_vitality -= damage
                        
        # game loop
        current_round = 1
        while not winner:
            if current_round > self.max_match_round:
                # if current round exceeds the max number of rounds per match
                # the player without the upper hand wins
                # the probability of going above the max round limit is rare
                if upper_hand == 'hero':
                    if verbose: print(f"After {self.max_match_round} rounds, {boss.name} wins.")
                    winner = 'boss'
                else:
                    if verbose: print(f"After {self.max_match_round} rounds, {self.hero.name} wins.")
                    winner = 'hero'
            else:
                current_round += 1
                if verbose: 
                    print("###", f"Round {current_round}", "###")
                    print(f"{self.hero.name} Health: {self.hero.current_vitality}, {boss.name} Health: {boss.current_vitality}")
                        
                if upper_hand == 'hero':
                    hero_attack()

                    # check boss health
                    if boss.current_vitality <= 0:
                        winner = 'hero'
                    else:
                        boss_attack()
                        # check hero health 
                        if self.hero.current_vitality <=0: winner = 'boss'
                else: # boss got upper hand 
                    boss_attack()
                    # check hero health 
                    if self.hero.current_vitality <=0: 
                        winner = 'boss'
                    else:
                        hero_attack()
                        if boss.current_vitality <= 0: winner = 'hero'

                # boss with 'self healing' ability
                if not winner and boss.perk == 'self_healing' and boss.current_vitality > 0:
                    boss.self_healing()
                    if verbose: print(f"{boss.name} uses self healing perk, HP + {boss.heals_per_round}")
                        
        if verbose: print("###End Match###")
        return winner
    
            
            
    # simulate the decision making process by taking user's input
    # input: player responses
    # output: game stats and hero progression
    def play(self, n_stages, auto=False, decisions=None):    
        # some stats
        mini_boss_defeated = 0
        champions_defeated = 0
        ultimate_defeated = 0
        
        
        # source: wikipedia
        boss_names = ["Brynhildr", "Eir", "Geirahöð", "Geiravör", "Geirdriful", "Geirönul",
                     "Geirskögul", "Göll", "Göndul", "Guðr", "Herfjötur", "Herja", "Hlaðguðr svanhvít",
                     "Hildr", "Hjalmþrimul", "Hervör alvitr", "Hjörþrimul", "Hlökk", "Hrist", "Hrund",
                     "Kára", "Mist", "Ölrún", "Randgríðr", "Ráðgríðr", "Reginleif", "Róta", "Sanngriðr",
                     "Sigrdrífa", "Sigrún", "Skalmöld", "Skeggöld", "Skögul", "Skuld", "Sveið",
                     "Svipul", "Þögn", "Þrima", "Þrúðr", "Skadi", "Freyja", "Brynhild", "Lagertha",
                     "Hervor", "Freydis Eiríksdóttir", "Gudrid Thorbjarnardóttir", "Sigrid the Proud",
                     "Unn the Deep-Minded", "Olga of Kiev", "Alva", "Asger", "Aslaug", "Astrid",
                     "Balder", "Bard", "Birger", "Bjarni", "Björn", "Brandt", "Brenda", "Brenna", 
                     "Calder", "Colby", "Corey", "Crosby", "Destin", "Eerika", "Egil", "Eira", "Eric",
                     "Erika", "Erlend", "Freja", "Gudmund", "Gustav", "Hagen", "Halvar", "Igor", "Kjeld",
                     "Langley", "Odin", "Olaf", "Olga", "Olsen", "Rangvaldr", "Sigfrid", "Thor", "Loki",
                     "Stig", "Sylvi", "Torbjörn", "Ulf", "Vernon",]
        

        for stage in range(1, n_stages+1):
            
            # begin stage
            self.prepare_stage()
            boss_name = np.random.choice(boss_names)
            
            
            print("*********************************************")
            print(f"Stage {stage}: *{self.hero.name}* Level {self.hero.level}, HP: {self.hero.current_vitality}")
            print(f"MAX_HP: {self.hero.max_vitality}, ATT: {self.hero.attack_power}, DEF: {self.hero.defense_power}")
            print("*********************************************")
            
            if auto:
                action = decisions[stage-1]  # use predefined strategy
            else:
                # calculate win rates
                mini_boss_win_rate, champion_win_rate, ult_boss_win_rate = self.hero.get_win_rates(200)
                
                print("Choose an action:")
                print("1: Restore Full Health")
                print("2: +30 Max HP")
                print("3: Attack +3")
                print("4: Defense + 3")
                print(f"5: Fight a mini boss (win rate: {round(mini_boss_win_rate * 100, 2)}%)")
                print(f"6: Fight a champion (win rate: {round(champion_win_rate * 100, 2)}%)")
                print(f"7: Fight an ultimate boss (win rate: {round(ult_boss_win_rate * 100, 2)}%)")
                print(f"8: Give up.. ")
                
                action = input()  # request user input
                print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
                
            if int(action) == 1:
                self.hero.restore_health("full")
                print(f"*{self.hero.name}* regains full HP.")
            elif int(action) == 2:
                self.hero.increase_max_vitality(points=30)
                print(f"*{self.hero.name}* max vitality + 30")
            elif int(action) == 3:
                self.hero.increase_attack_power(points=3)
                print(f"*{self.hero.name}* attack power + 3")
            elif int(action) == 4:
                self.hero.increase_defense_power(points=3)
                print(f"*{self.hero.name}* defense power + 3")
            elif int(action) == 5:
                if auto:
                    print(f"*{self.hero.name}* vs. mini boss {boss_name}")
                    winner = self.match(boss_name, "mini", verbose=False)
                else:
                    winner = self.match(boss_name, "mini", verbose=True)
                
                if winner == 'hero':
                    print(f"*{self.hero.name}* wins.")
                    mini_boss_defeated += 1
                    print(f"*{self.hero.name}* level +1.")
                    self.hero.level_up()
                else:
                    print(f"{boss_name} wins.")
            elif int(action) == 6:              
                if auto:
                    print(f"*{self.hero.name}* vs. champion {boss_name}")
                    winner = self.match(boss_name, "champion", verbose=False)
                else:
                    winner = self.match(boss_name, "champion", verbose=True)

                if winner == 'hero':
                    print(f"*{self.hero.name}* wins.")
                    champions_defeated += 1
                    print(f"*{self.hero.name}* level +2.")
                    self.hero.level_up(lv=2)
                else:
                    print(f"{boss_name} wins.")
            elif int(action) == 7:              
                if auto:
                    print(f"*{self.hero.name}* vs. ultimate boss {boss_name}")
                    winner = self.match(boss_name, "ultimate", verbose=False)
                else:
                    winner = self.match(boss_name, "ultimate", verbose=True)
              
                if winner == 'hero':
                    print(f"*{self.hero.name}* wins.")
                    ultimate_defeated += 1
                    print(f"*{self.hero.name}* level +3.")
                    self.hero.level_up(lv=3)
                else:
                    print(f"{boss_name} wins.")
            else:
                print(f"There is no failure except in no longer trying.") 
                print(f"Adieu *{self.hero.name}*")
                break
               
            # for readability 
            if auto: print("\n")
        
        print("*********************************************")
        print("** Final stage **")
        print(f"Hero *{self.hero.name}* has reached level {self.hero.level}! HP: {max(0, self.hero.current_vitality)}")
        print(f"Final MAX_HP: {self.hero.max_vitality}, ATT: {self.hero.attack_power}, DEF: {self.hero.defense_power}")
        print(f"-Mini bosses defeated: {mini_boss_defeated}")
        print(f"-Champions defeated: {champions_defeated}")
        print(f"-Ultimate bosses defeated: {ultimate_defeated}")
        print("~Thanks for playing. TG~")
        print("*********************************************")
        self.reset_hero()
        
    # call to initialize each stage
    def prepare_stage(self):
        if self.hero.current_vitality <= 0: self.hero.restore_health("half")
            
    
    def reset_hero(self):
        self.hero.reborn()
        
        
# driver function to run the simulation 
# allows user input
def run_game():
    print("Enter hero name: ")
    hero_name = input()
    hero = Hero(hero_name)
    
    print("Enter the number of days to train and level up: (DIGITS ONLY)")
    n_stages = int(input())

    print("Starting game...")
    game = Game(hero)
    game.play(n_stages)

Monte Carlo Test
=========

Before we start on the dynamic solutions, we experiement and test the accuracy of monte carlo simulation simulating \omega_k.

Instantiate a hero of stats:
1. max_hp = 500
2. cur_hp = 500
3. att = 50
4. def = 50

Boss level: ultimate

In [7]:
hero = Hero("MC")
hero.any_stats(1, 500, 500, 50, 50)
boss_level = "ultimate"

# n_trials = 10   
trial_10_results = []
for _ in range(10):
    win_rate, avg_hp = hero.trial(10, boss_level)
    trial_10_results.append((win_rate, avg_hp))
    
# average out:
win_rates_10 = [test[0] for test in trial_10_results]
mean_10 = np.mean(win_rates_10)
var_10 = np.var(win_rates_10)

# n_trials = 50
trial_50_results = []
for _ in range(10):
    win_rate, avg_hp = hero.trial(50, boss_level)
    trial_50_results.append((win_rate, avg_hp))
    
# average out:
win_rates_50 = [test[0] for test in trial_50_results]
mean_50 = np.mean(win_rates_50)
var_50 = np.var(win_rates_50)

# n_trials = 100
trial_100_results = []
for _ in range(10):
    win_rate, avg_hp = hero.trial(100, boss_level)
    trial_100_results.append((win_rate, avg_hp))
    
# average out:
win_rates_100 = [test[0] for test in trial_100_results]
mean_100 = np.mean(win_rates_100)
var_100 = np.var(win_rates_100)

# n_trials = 200
trial_200_results = []
for _ in range(10):
    win_rate, avg_hp = hero.trial(200, boss_level)
    trial_200_results.append((win_rate, avg_hp))

# average out:
win_rates_200 = [test[0] for test in trial_200_results]
mean_200 = np.mean(win_rates_200)
var_200 = np.var(win_rates_200)

# n_trials = 500
trial_500_results = []
for _ in range(10):
    win_rate, avg_hp = hero.trial(500, boss_level)
    trial_500_results.append((win_rate, avg_hp))

# average out:
win_rates_500 = [test[0] for test in trial_500_results]
mean_500 = np.mean(win_rates_500)
var_500 = np.var(win_rates_500)

# n_trials = 1000
trial_1000_results = []
for _ in range(10):
    win_rate, avg_hp = hero.trial(1000, boss_level)
    trial_1000_results.append((win_rate, avg_hp))

# average out:
win_rates_1000 = [test[0] for test in trial_1000_results]
mean_1000 = np.mean(win_rates_1000)
var_1000 = np.var(win_rates_1000)

MC test results

In [8]:
print("N_trials = 10, MC accuracy test results: (win_rate, avg_hp)")
print(trial_10_results, "avg: ", round(mean_10, 2), "var: ", var_10)
print()

print("N_trials = 50, MC accuracy test results: (win_rate, avg_hp)")
print(trial_50_results, "avg: ", round(mean_50, 2), "var: ", var_50)
print()

print("N_trials = 100, MC accuracy test results: (win_rate, avg_hp)")
print(trial_100_results, "avg: ", round(mean_100, 2), "var: ", var_100)
print()

print("N_trials = 200, MC accuracy test results: (win_rate, avg_hp)")
print(trial_200_results, "avg: ", round(mean_200, 2), "var: ", var_200)
print()

print("N_trials = 500, MC accuracy test results: (win_rate, avg_hp)")
print(trial_500_results, "avg: ", round(mean_500, 2), "var: ", var_500)
print()

print("N_trials = 1000, MC accuracy test results: (win_rate, avg_hp)")
print(trial_1000_results, "avg: ", round(mean_1000, 2), "var: ", var_1000)

N_trials = 10, MC accuracy test results: (win_rate, avg_hp)
[(0.8, 110), (0.8, 70), (0.5, 110), (0.6, 100), (0.5, 90), (0.6, 100), (0.8, 70), (1.0, 110), (0.7, 90), (0.7, 110)] avg:  0.7 var:  0.022000000000000002

N_trials = 50, MC accuracy test results: (win_rate, avg_hp)
[(0.64, 110), (0.66, 90), (0.7, 90), (0.78, 120), (0.68, 90), (0.74, 100), (0.8, 100), (0.78, 100), (0.62, 110), (0.62, 90)] avg:  0.7 var:  0.004276000000000001

N_trials = 100, MC accuracy test results: (win_rate, avg_hp)
[(0.72, 90), (0.74, 100), (0.68, 100), (0.73, 100), (0.74, 100), (0.71, 100), (0.71, 100), (0.79, 110), (0.74, 90), (0.75, 110)] avg:  0.73 var:  0.0007690000000000003

N_trials = 200, MC accuracy test results: (win_rate, avg_hp)
[(0.705, 100), (0.735, 100), (0.755, 100), (0.71, 110), (0.72, 100), (0.675, 110), (0.73, 100), (0.685, 100), (0.695, 100), (0.68, 110)] avg:  0.71 var:  0.0006139999999999992

N_trials = 500, MC accuracy test results: (win_rate, avg_hp)
[(0.702, 100), (0.656, 110), (0.7

Heuristic
=========

Basic strategy:
1. Fight the highest level boss with a win rate more than 80% 
2. If injured, restore full health immediately 
3. Otherwise keep increasing max vitality

In [8]:
def Heuristic(game, n_stages, n_trials=100, win_rates=None, check_db=False):

    
    def process(win_rates=None):
        decisions = []
        
        for stage in range(1, n_stages+1):
            print(f"optimizing stage {stage}", end="\r")

            game.prepare_stage()
                
            mini_boss_win_rate, champion_win_rate, ult_boss_win_rate \
            = game.hero.get_win_rates(n_trials, win_rates)

            if ult_boss_win_rate >= 0.8:
                # simulate a match with an ultimate boss
                winner = game.match('h_boss_ult', "ultimate", verbose=False)
                if winner == 'hero': game.hero.level_up(lv=3)
                decisions.append(7)
            elif champion_win_rate >= 0.8:
                # simulate a match with a champion level boss
                winner = game.match('h_boss_champ', "champion", verbose=False)
                if winner == 'hero': game.hero.level_up(lv=2)
                decisions.append(6)
            elif mini_boss_win_rate >= 0.8:
                # simulate a match with a mini boss
                winner = game.match('h_boss_mini', "mini", verbose=False)
                if winner == 'hero': game.hero.level_up(lv=1)
                decisions.append(5)
            elif game.hero.current_vitality < game.hero.max_vitality:
                # restore full health
                game.hero.restore_health("full")
                decisions.append(1)
            else:
                # increase max vitality
                game.hero.increase_max_vitality(points=30)
                decisions.append(2)
        
        # return endgame hero level and all decision steps by heuristic
        return (game.hero.level, decisions,)
        
    # select which win rate dict to use 
    if win_rates is not None:
        db = win_rates
    elif check_db:
        db = db_win_rate.db
    else:
        db = None
    
    exp_lv, steps = process(db)
    
    # update database
    if win_rates is not None:
        win_rates.update(db)
    elif check_db:
        db_win_rate.db.update(db)
        
    return exp_lv, steps

Try the heuristic algorithm
=====================

First let's explore a simple 10 stage game 

In [178]:
n_stages = 10
hero = Hero("heuristic")
game = Game(hero)
lv_h10, steps_h10, = Heuristic(game, n_stages, n_trials=1000, check_db=True)

print("Heuristic algorithm")
print(f"Expected level: {lv_h10}")
print(f"Optimal steps: {steps_h10}")

Heuristic algorithm
Expected level: 4
Optimal steps: [2, 2, 2, 2, 5, 1, 5, 1, 5, 1]


We saved the calculated win rates to the win rate database. Visualize it below

Format: (Current HP/Att/Def, Boss_Level) : (Win Rate, Avg HP Left After Win)

(If the chance of winning is 0, then the Avg HP left is simply assigned to 1)

In [11]:
print("Number of win rates in dict: ", len(db_win_rate.db), "\n")
print(db_win_rate.db)

Number of win rates in dict:  30 

{('100/20/10', 'champion'): (0.0, 1), ('100/20/10', 'mini'): (0.031, 10), ('100/20/10', 'ultimate'): (0.0, 1), ('130/20/10', 'champion'): (0.0, 1), ('130/20/10', 'mini'): (0.178, 20), ('130/20/10', 'ultimate'): (0.0, 1), ('160/20/10', 'champion'): (0.0, 1), ('160/20/10', 'mini'): (0.431, 30), ('160/20/10', 'ultimate'): (0.0, 1), ('190/20/10', 'champion'): (0.0, 1), ('190/20/10', 'mini'): (0.614, 40), ('190/20/10', 'ultimate'): (0.0, 1), ('220/20/10', 'champion'): (0.0, 1), ('220/20/10', 'mini'): (0.81, 60), ('220/20/10', 'ultimate'): (0.0, 1), ('107/21/11', 'champion'): (0.0, 1), ('107/21/11', 'mini'): (0.107, 10), ('107/21/11', 'ultimate'): (0.0, 1), ('230/21/11', 'champion'): (0.0, 1), ('230/21/11', 'mini'): (0.905, 80), ('230/21/11', 'ultimate'): (0.0, 1), ('117/22/12', 'champion'): (0.0, 1), ('117/22/12', 'mini'): (0.238, 20), ('117/22/12', 'ultimate'): (0.0, 1), ('240/22/12', 'champion'): (0.0, 1), ('240/22/12', 'mini'): (0.969, 100), ('240/22/12

Detail the result for Heuristic, N=10:

In [12]:
game.reset_hero()
game.play(n_stages, auto=True, decisions=steps_h10)

*********************************************
Stage 1: *heuristic* Level 1, HP: 100
MAX_HP: 100, ATT: 20, DEF: 10
*********************************************
*heuristic* max vitality + 30


*********************************************
Stage 2: *heuristic* Level 1, HP: 130
MAX_HP: 130, ATT: 20, DEF: 10
*********************************************
*heuristic* max vitality + 30


*********************************************
Stage 3: *heuristic* Level 1, HP: 160
MAX_HP: 160, ATT: 20, DEF: 10
*********************************************
*heuristic* max vitality + 30


*********************************************
Stage 4: *heuristic* Level 1, HP: 190
MAX_HP: 190, ATT: 20, DEF: 10
*********************************************
*heuristic* max vitality + 30


*********************************************
Stage 5: *heuristic* Level 1, HP: 220
MAX_HP: 220, ATT: 20, DEF: 10
*********************************************
*heuristic* vs. mini boss Freyja
*heuristic* wins.
*heuristic* level +1.

Try a larger instance of Heuristic, N=25

As we have computed most of the win rates for N=10 before, 

the program will swiftly go through all the previously visited states

without recomputing the transition probability

In [13]:
n_stages = 25
game.reset_hero()
lv_h25, steps_h25 = Heuristic(game, n_stages, n_trials=1000, check_db=True)
print(f"Heuristic algorithm, {n_stages} stages")
print(f"Expected level: {lv_h25}")
print(f"Optimal steps: {steps_h25}")

Heuristic algorithm, 25 stages
Expected level: 13
Optimal steps: [2, 2, 2, 2, 5, 1, 5, 1, 5, 1, 5, 1, 5, 1, 5, 5, 1, 5, 5, 1, 5, 5, 1, 5, 5]


In [14]:
# detail the above result
game.reset_hero()
game.play(n_stages, auto=True, decisions=steps_h25)

*********************************************
Stage 1: *heuristic* Level 1, HP: 100
MAX_HP: 100, ATT: 20, DEF: 10
*********************************************
*heuristic* max vitality + 30


*********************************************
Stage 2: *heuristic* Level 1, HP: 130
MAX_HP: 130, ATT: 20, DEF: 10
*********************************************
*heuristic* max vitality + 30


*********************************************
Stage 3: *heuristic* Level 1, HP: 160
MAX_HP: 160, ATT: 20, DEF: 10
*********************************************
*heuristic* max vitality + 30


*********************************************
Stage 4: *heuristic* Level 1, HP: 190
MAX_HP: 190, ATT: 20, DEF: 10
*********************************************
*heuristic* max vitality + 30


*********************************************
Stage 5: *heuristic* Level 1, HP: 220
MAX_HP: 220, ATT: 20, DEF: 10
*********************************************
*heuristic* vs. mini boss Alva
*heuristic* wins.
*heuristic* level +1.



In [ ]:
print

Now try a even larger instance of Heuristic, N=50

Using the saved dict there is little need for recomputation for state 1-25

In [15]:
n_stages = 50
game.reset_hero()
lv_h50, steps_h50 = Heuristic(game, n_stages, n_trials=1000, check_db=True)
print(f"Heuristic algorithm, {n_stages} stages")
print(f"Expected level: {lv_h50}")
print(f"Optimal steps: {steps_h50}")

Heuristic algorithm, 50 stages
Expected level: 42
Optimal steps: [2, 2, 2, 2, 5, 1, 5, 1, 5, 1, 5, 1, 5, 5, 1, 5, 5, 1, 5, 5, 1, 5, 5, 5, 1, 5, 5, 5, 5, 5, 5, 1, 6, 6, 5, 5, 5, 1, 6, 6, 5, 5, 5, 5, 5, 5, 1, 7, 6, 1]


In [16]:
game.reset_hero()
game.play(n_stages, auto=True, decisions=steps_h50)

*********************************************
Stage 1: *heuristic* Level 1, HP: 100
MAX_HP: 100, ATT: 20, DEF: 10
*********************************************
*heuristic* max vitality + 30


*********************************************
Stage 2: *heuristic* Level 1, HP: 130
MAX_HP: 130, ATT: 20, DEF: 10
*********************************************
*heuristic* max vitality + 30


*********************************************
Stage 3: *heuristic* Level 1, HP: 160
MAX_HP: 160, ATT: 20, DEF: 10
*********************************************
*heuristic* max vitality + 30


*********************************************
Stage 4: *heuristic* Level 1, HP: 190
MAX_HP: 190, ATT: 20, DEF: 10
*********************************************
*heuristic* max vitality + 30


*********************************************
Stage 5: *heuristic* Level 1, HP: 220
MAX_HP: 220, ATT: 20, DEF: 10
*********************************************
*heuristic* vs. mini boss Herja
*heuristic* wins.
*heuristic* level +1.


In [37]:
print(db_win_rate.db[('570/55/45', 'ultimate')])

(0.94, 160)


check the win rate database again

In [17]:
print("Number of win rates in dict: ", len(db_win_rate.db), "\n")
print(db_win_rate.db)

Number of win rates in dict:  195 

{('100/20/10', 'champion'): (0.0, 1), ('100/20/10', 'mini'): (0.031, 10), ('100/20/10', 'ultimate'): (0.0, 1), ('130/20/10', 'champion'): (0.0, 1), ('130/20/10', 'mini'): (0.178, 20), ('130/20/10', 'ultimate'): (0.0, 1), ('160/20/10', 'champion'): (0.0, 1), ('160/20/10', 'mini'): (0.431, 30), ('160/20/10', 'ultimate'): (0.0, 1), ('190/20/10', 'champion'): (0.0, 1), ('190/20/10', 'mini'): (0.614, 40), ('190/20/10', 'ultimate'): (0.0, 1), ('220/20/10', 'champion'): (0.0, 1), ('220/20/10', 'mini'): (0.81, 60), ('220/20/10', 'ultimate'): (0.0, 1), ('107/21/11', 'champion'): (0.0, 1), ('107/21/11', 'mini'): (0.107, 10), ('107/21/11', 'ultimate'): (0.0, 1), ('230/21/11', 'champion'): (0.0, 1), ('230/21/11', 'mini'): (0.905, 80), ('230/21/11', 'ultimate'): (0.0, 1), ('117/22/12', 'champion'): (0.0, 1), ('117/22/12', 'mini'): (0.238, 20), ('117/22/12', 'ultimate'): (0.0, 1), ('240/22/12', 'champion'): (0.0, 1), ('240/22/12', 'mini'): (0.969, 100), ('240/22/1

Rollout
=========

Strategy:
1. Based on the base heuristic
2. At each stage: for each available action u_k(x_k)
    simulate the respective Q factor by Monte Carlo
3. In other words, choose the very best action with the maximum expected return (level) 
    for a given state x_k, approximate the max return from x_{k+1} using the heuristic

In [20]:
# This version of Rollout checks saved state dict to reduce CPU computation
# However the bottleneck will be the shared Python dictionary
# Across multiple threads
# For faster CPUs, set use_db to False

def Rollout(game, n_stages, n_trials=100, q=21, use_db=False): #q: number of trajectory to simulate 
    
        
    # following script requires multi processing
    # stage: current_stage i the parent process is at
    # manager_dict: multiprocessing.Manager().dict() to save action result
    # win_rates: known pre-computed win rates to speed up the process for some instances
    def worker_1(stage, manager_dict, win_rates_db=None):
        Q_exp_final_lv = 0
        Q_game = deepcopy(game)
        
        if game.hero.current_vitality < game.hero.max_vitality: 
            
            Q_game.hero.restore_health("full")
            
            for _ in range(q):
                print(f"Worker 1: simulating q: {_+1}/{q}", end="\r")

                tmp_game = deepcopy(Q_game)

                # calculate expected level increase for stage k+1 using the base heuristic
                final_lv, steps = Heuristic(tmp_game, n_stages - stage, n_trials, win_rates_db)
                Q_exp_final_lv += final_lv

            # final Q factor
            Q_exp_final_lv = round(Q_exp_final_lv/q, 2)
        
        log = f"Action 1, estimated Q value (expected final level): {Q_exp_final_lv}"
        
        manager_dict["worker_1"] = (Q_exp_final_lv, Q_game, log)


    def worker_2(stage, manager_dict, win_rates_db=None):
        Q_game = deepcopy(game)
        Q_game.hero.increase_max_vitality(points=30)

        Q_exp_final_lv = 0
        
        for _ in range(q):
            print(f"Worker 2: simulating q: {_+1}/{q}", end="\r")

            tmp_game = deepcopy(Q_game)

            # calculate expected level increase for stage k+1 using the base heuristic
            final_lv, steps, = Heuristic(tmp_game, n_stages - stage, n_trials, win_rates_db)
            Q_exp_final_lv += final_lv

        # final Q factor
        Q_exp_final_lv = round(Q_exp_final_lv/q, 2)
        log = f"Action 2, estimated Q value (expected final level): {Q_exp_final_lv}"

        manager_dict["worker_2"] = (Q_exp_final_lv, Q_game, log)
        

    def worker_3(stage, manager_dict, win_rates_db=None):
        Q_game = deepcopy(game)
        Q_game.hero.increase_attack_power(points=3)

        Q_exp_final_lv = 0
        
        for _ in range(q):
            print(f"Worker 3: simulating q: {_+1}/{q}", end="\r")

            tmp_game = deepcopy(Q_game)

            # calculate expected level increase for stage k+1 using the base heuristic
            final_lv, steps, = Heuristic(tmp_game, n_stages - stage, n_trials, win_rates_db)
            Q_exp_final_lv += final_lv

        # final Q factor
        Q_exp_final_lv = round(Q_exp_final_lv/q, 2)
        log = f"Action 3, estimated Q value (expected final level): {Q_exp_final_lv}"

        manager_dict["worker_3"] = (Q_exp_final_lv, Q_game, log)


    def worker_4(stage, manager_dict, win_rates_db=None):
        Q_game = deepcopy(game)
        Q_game.hero.increase_defense_power(points=3)

        Q_exp_final_lv = 0
        
        for _ in range(q):
            print(f"Worker 4: simulating q: {_+1}/{q}", end="\r")

            tmp_game = deepcopy(Q_game)

            # calculate expected level increase for stage k+1 using the base heuristic
            final_lv, steps, = Heuristic(tmp_game, n_stages - stage, n_trials, win_rates_db)
            Q_exp_final_lv += final_lv

        # final Q factor
        Q_exp_final_lv = round(Q_exp_final_lv/q, 2)
        log = f"Action 4, estimated Q value (expected final level): {Q_exp_final_lv}"

        manager_dict["worker_4"] = (Q_exp_final_lv, Q_game, log)
        

    def worker_5(stage, manager_dict, win_rates_db=None):
        Q_game = deepcopy(game)
        Q_exp_final_lv = 0
        
        for _ in range(q):
            print(f"Worker 5 simulating q: {_+1}/{q}", end="\r")

            tmp_game = deepcopy(Q_game)

            winner = tmp_game.match("mini_boss", "mini", verbose=False)

            if winner == 'hero': tmp_game.hero.level_up()

            # calculate expected level increase for stage k+1 using the base heuristic
            final_lv, steps,= Heuristic(tmp_game, n_stages - stage, n_trials, win_rates_db)
            Q_exp_final_lv += final_lv

        
        # final Q factor
        Q_exp_final_lv = round(Q_exp_final_lv/q, 2)

        win_rate, avg_hp = Q_game.hero.trial(100, "mini", win_rates_db)
        
        # hero level up by exp. level increase (win_rate * 1)
        Q_game.hero.current_vitality = avg_hp
        Q_game.hero.level_up(round(win_rate, 2))

        log = f"Action 5, estimated Q value (expected final level): {Q_exp_final_lv}"
        
        manager_dict["worker_5"] = (Q_exp_final_lv, Q_game, log)


    def worker_6(stage, manager_dict, win_rates_db=None):

        Q_game = deepcopy(game)
        Q_exp_final_lv = 0

        for _ in range(q):
            print(f"Worker 6 simulating q: {_+1}/{q}", end="\r")
            
            tmp_game = deepcopy(Q_game)

            winner = tmp_game.match("champ", "champion", verbose=False)

            if winner == 'hero': tmp_game.hero.level_up(lv=2)

            # calculate expected level increase for stage k+1 using the base heuristic
            final_lv, steps, = Heuristic(tmp_game, n_stages - stage, n_trials, win_rates_db)
            Q_exp_final_lv += final_lv

        # final Q factor
        Q_exp_final_lv = round(Q_exp_final_lv/q, 2)
        
        win_rate, avg_hp = Q_game.hero.trial(100, "champion", win_rates_db)
        
        # hero level up by exp. level increase (win_rate * 2)
        Q_game.hero.current_vitality = avg_hp
        Q_game.hero.level_up(round(win_rate * 2, 2))

        log = f"Action 6, estimated Q value (expected final level): {Q_exp_final_lv}"
        
        manager_dict["worker_6"] = (Q_exp_final_lv, Q_game, log)


    def worker_7(stage, manager_dict, win_rates_db=None):
        Q_game = deepcopy(game)
        Q_exp_final_lv = 0

        for _ in range(q):
            print(f"Simulating q: {_+1}/{q}", end="\r")

            tmp_game = deepcopy(Q_game)

            winner = tmp_game.match("ult", "ultimate", verbose=False)

            if winner == 'hero': tmp_game.hero.level_up(lv=3)

            # calculate expected level increase for stage k+1 using the base heuristic
            final_lv, steps, = Heuristic(tmp_game, n_stages - stage, n_trials, win_rates_db)
            Q_exp_final_lv += final_lv

        # final Q factor
        Q_exp_final_lv = round(Q_exp_final_lv/q, 2)
        
        win_rate, avg_hp = Q_game.hero.trial(100, "ultimate", win_rates_db)
        # hero level up by exp. level increase (win_rate * 3)
        Q_game.hero.current_vitality = avg_hp
        Q_game.hero.level_up(round(win_rate * 3, 2))
        
        log = f"Action 7, estimated Q value (expected final level): {Q_exp_final_lv}"
        
        manager_dict["worker_7"] = (Q_exp_final_lv, Q_game, log)


    decisions = []
    manager = multiprocessing.Manager()
    
    for stage in range(1, n_stages+1):  # 1.. k-1 stages
        
        # initialize stage
        game.prepare_stage()
        
        print(f"*Stage {stage}*")
        #print("-------------------------------------------------------------------")
        best_action = None
        best_game = None
        max_exp_level = -np.inf
        
        manager_dict = manager.dict()
        
        # if requested to check the win rate database
        if use_db:
            db = db_win_rate.db
            win_rates_db = manager.dict(db)
        else:
            win_rates_db = None
            
        targets = [worker_1, worker_2, worker_3, worker_4, 
                   worker_5, worker_6, worker_7]
        
        jobs = []
        
        for i in range(len(targets)):
            p = multiprocessing.Process(target=targets[i], args=(stage, manager_dict, win_rates_db))
            jobs.append(p)
            p.start()
            
        for proc in jobs:
            proc.join()
        
        for action in range(1, 8):
            key = f"worker_{action}"
            exp_lv, game, log = manager_dict[key]
            
            print(log)
            
            # if action returns max exp. final lv
            if exp_lv > max_exp_level:
                max_exp_level = exp_lv
                best_action = action
                best_game = game
           
        
        print(f"Stage {stage} optimal action: {best_action} \n")
        decisions.append(best_action)
        
        game = best_game  # use the optimal game and hero stats

        # update the database
        if use_db: db.update(win_rates_db)
            
    return game.hero.level, decisions
                

Try out the rollout algorithm
=====================

First try optimizing a small instance of 10 stages via rollout

In [21]:
n_stages = 10
hero = Hero("Rollout")
game = Game(hero)
# note: if use_db is toggled false
# cpu may max out!
lv_r10, steps_r10 = Rollout(game, n_stages, q=51, use_db=False)
print(f"Rollout algorithm, instance {n_stages}")
print(f"Expected final level: {lv_r10}")
print(f"Optimal steps: {steps_r10}")

*Stage 1*
Action 1, estimated Q value (expected final level): 0
Action 2, estimated Q value (expected final level): 3.76
Action 3, estimated Q value (expected final level): 3.8
Action 4, estimated Q value (expected final level): 3.75
Action 5, estimated Q value (expected final level): 2.88
Action 6, estimated Q value (expected final level): 2.78
Action 7, estimated Q value (expected final level): 2.73
Stage 1 optimal action: 3 

*Stage 2*
Action 1, estimated Q value (expected final level): 0
Action 2, estimated Q value (expected final level): 3.8
Action 3, estimated Q value (expected final level): 3.88
Action 4, estimated Q value (expected final level): 3.84
Action 5, estimated Q value (expected final level): 2.96
Action 6, estimated Q value (expected final level): 2.82
Action 7, estimated Q value (expected final level): 2.88
Stage 2 optimal action: 3 

*Stage 3*
Action 1, estimated Q value (expected final level): 0
Action 2, estimated Q value (expected final level): 3.88
Action 3, est

In [22]:
# detail the above result
game.reset_hero()
game.play(n_stages, auto=True, decisions=steps_r10)

*********************************************
Stage 1: *Rollout* Level 1, HP: 100
MAX_HP: 100, ATT: 20, DEF: 10
*********************************************
*Rollout* attack power + 3


*********************************************
Stage 2: *Rollout* Level 1, HP: 100
MAX_HP: 100, ATT: 23, DEF: 10
*********************************************
*Rollout* attack power + 3


*********************************************
Stage 3: *Rollout* Level 1, HP: 100
MAX_HP: 100, ATT: 26, DEF: 10
*********************************************
*Rollout* max vitality + 30


*********************************************
Stage 4: *Rollout* Level 1, HP: 130
MAX_HP: 130, ATT: 26, DEF: 10
*********************************************
*Rollout* max vitality + 30


*********************************************
Stage 5: *Rollout* Level 1, HP: 160
MAX_HP: 160, ATT: 26, DEF: 10
*********************************************
*Rollout* max vitality + 30


*********************************************
Stage 6: *Rollou

Try instantce Rollout, N=25

In [23]:
game.reset_hero()
n_stages = 25
lv_r25, steps_r25 = Rollout(game, n_stages, q=24, use_db=False)
print(f"Rollout algorithm, instance {n_stages}")
print(f"Expected final level: {lv_r25}")
print(f"Optimal steps: {steps_r25}")

*Stage 1*
Action 1, estimated Q value (expected final level): 0
Action 2, estimated Q value (expected final level): 13.58
Action 3, estimated Q value (expected final level): 14.08
Action 4, estimated Q value (expected final level): 14.17
Action 5, estimated Q value (expected final level): 12.38
Action 6, estimated Q value (expected final level): 12.21
Action 7, estimated Q value (expected final level): 12.21
Stage 1 optimal action: 4 

*Stage 2*
Action 1, estimated Q value (expected final level): 0
Action 2, estimated Q value (expected final level): 14.17
Action 3, estimated Q value (expected final level): 14.12
Action 4, estimated Q value (expected final level): 13.75
Action 5, estimated Q value (expected final level): 12.25
Action 6, estimated Q value (expected final level): 12.75
Action 7, estimated Q value (expected final level): 12.67
Stage 2 optimal action: 2 

*Stage 3*
Action 1, estimated Q value (expected final level): 0
Action 2, estimated Q value (expected final level): 14.1

Action 1, estimated Q value (expected final level): 0
Action 2, estimated Q value (expected final level): 13.4
Action 3, estimated Q value (expected final level): 13.36
Action 4, estimated Q value (expected final level): 13.28
Action 5, estimated Q value (expected final level): 14.24
Action 6, estimated Q value (expected final level): 13.9
Action 7, estimated Q value (expected final level): 12.82
Stage 20 optimal action: 5 

*Stage 21*
Action 1, estimated Q value (expected final level): 13.82
Action 2, estimated Q value (expected final level): 13.24
Action 3, estimated Q value (expected final level): 13.4
Action 4, estimated Q value (expected final level): 13.44
Action 5, estimated Q value (expected final level): 14.4
Action 6, estimated Q value (expected final level): 13.82
Action 7, estimated Q value (expected final level): 13.15
Stage 21 optimal action: 5 

*Stage 22*
Action 1, estimated Q value (expected final level): 13.82
Action 2, estimated Q value (expected final level): 13.19


In [23]:
print(db_win_rate.db[('350/33/23', 'champion')])

(0.82, 90)


In [24]:
# detail the above result
game.reset_hero()
game.play(n_stages, auto=True, decisions=steps_r25)

*********************************************
Stage 1: *Rollout* Level 1, HP: 100
MAX_HP: 100, ATT: 20, DEF: 10
*********************************************
*Rollout* defense power + 3


*********************************************
Stage 2: *Rollout* Level 1, HP: 100
MAX_HP: 100, ATT: 20, DEF: 13
*********************************************
*Rollout* max vitality + 30


*********************************************
Stage 3: *Rollout* Level 1, HP: 130
MAX_HP: 130, ATT: 20, DEF: 13
*********************************************
*Rollout* max vitality + 30


*********************************************
Stage 4: *Rollout* Level 1, HP: 160
MAX_HP: 160, ATT: 20, DEF: 13
*********************************************
*Rollout* max vitality + 30


*********************************************
Stage 5: *Rollout* Level 1, HP: 190
MAX_HP: 190, ATT: 20, DEF: 13
*********************************************
*Rollout* max vitality + 30


*********************************************
Stage 6: *Roll

Confidence level (win rate) for fighting a champion

In [21]:
print(db_win_rate.db[('350/33/23', "champion")])

(0.82, 90)


Finally, approximate a large instance of Rollout, N=50

In [33]:
n_stages = 50
game.reset_hero()
# CPU will max out!
lv_r50, steps_r50 = Rollout(game, n_stages, use_db=False)
print(f"Rollout algorithm, instance {n_stages}")
print(f"Expectef final level: {lv_r50}")
print(f"Optimal steps: {steps_r50}")

*Stage 1*
Action 1, estimated Q value (expected final level): 0
Action 2, estimated Q value (expected final level): 42.62
Action 3, estimated Q value (expected final level): 44.0
Action 4, estimated Q value (expected final level): 43.43
Action 5, estimated Q value (expected final level): 41.81
Action 6, estimated Q value (expected final level): 40.81
Action 7, estimated Q value (expected final level): 40.95
Stage 1 optimal action: 3 

*Stage 2*
Action 1, estimated Q value (expected final level): 0
Action 2, estimated Q value (expected final level): 44.0
Action 3, estimated Q value (expected final level): 43.71
Action 4, estimated Q value (expected final level): 44.29
Action 5, estimated Q value (expected final level): 41.24
Action 6, estimated Q value (expected final level): 41.38
Action 7, estimated Q value (expected final level): 40.9
Stage 2 optimal action: 4 

*Stage 3*
Action 1, estimated Q value (expected final level): 0
Action 2, estimated Q value (expected final level): 44.29
A

Action 1, estimated Q value (expected final level): 43.86
Action 2, estimated Q value (expected final level): 43.53
Action 3, estimated Q value (expected final level): 43.91
Action 4, estimated Q value (expected final level): 43.82
Action 5, estimated Q value (expected final level): 44.2
Action 6, estimated Q value (expected final level): 43.39
Action 7, estimated Q value (expected final level): 43.39
Stage 20 optimal action: 5 

*Stage 21*
Action 1, estimated Q value (expected final level): 43.77
Action 2, estimated Q value (expected final level): 44.15
Action 3, estimated Q value (expected final level): 43.72
Action 4, estimated Q value (expected final level): 43.29
Action 5, estimated Q value (expected final level): 45.15
Action 6, estimated Q value (expected final level): 43.48
Action 7, estimated Q value (expected final level): 43.82
Stage 21 optimal action: 5 

*Stage 22*
Action 1, estimated Q value (expected final level): 44.25
Action 2, estimated Q value (expected final level):

Action 1, estimated Q value (expected final level): 0
Action 2, estimated Q value (expected final level): 42.81
Action 3, estimated Q value (expected final level): 41.33
Action 4, estimated Q value (expected final level): 42.04
Action 5, estimated Q value (expected final level): 43.43
Action 6, estimated Q value (expected final level): 44.33
Action 7, estimated Q value (expected final level): 42.71
Stage 39 optimal action: 6 

*Stage 40*
Action 1, estimated Q value (expected final level): 42.38
Action 2, estimated Q value (expected final level): 42.04
Action 3, estimated Q value (expected final level): 42.47
Action 4, estimated Q value (expected final level): 42.85
Action 5, estimated Q value (expected final level): 44.09
Action 6, estimated Q value (expected final level): 44.85
Action 7, estimated Q value (expected final level): 43.04
Stage 40 optimal action: 6 

*Stage 41*
Action 1, estimated Q value (expected final level): 43.38
Action 2, estimated Q value (expected final level): 42

In [34]:
# detail the above result
game.reset_hero()
game.play(n_stages, auto=True, decisions=steps_r50)

*********************************************
Stage 1: *Rollout* Level 1, HP: 100
MAX_HP: 100, ATT: 20, DEF: 10
*********************************************
*Rollout* attack power + 3


*********************************************
Stage 2: *Rollout* Level 1, HP: 100
MAX_HP: 100, ATT: 23, DEF: 10
*********************************************
*Rollout* defense power + 3


*********************************************
Stage 3: *Rollout* Level 1, HP: 100
MAX_HP: 100, ATT: 23, DEF: 13
*********************************************
*Rollout* defense power + 3


*********************************************
Stage 4: *Rollout* Level 1, HP: 100
MAX_HP: 100, ATT: 23, DEF: 16
*********************************************
*Rollout* max vitality + 30


*********************************************
Stage 5: *Rollout* Level 1, HP: 130
MAX_HP: 130, ATT: 23, DEF: 16
*********************************************
*Rollout* max vitality + 30


*********************************************
Stage 6: *Rollo

In [40]:
print(db_win_rate.db[('550/67/54', 'ultimate')])

(1.0, 260)


Optimal Solution
===============

In [7]:
class Forward_recursion:
    def __init__(self, hero, n_stages, state_dict, n_trials=200):
        self.hero = hero
        self.n_stages = n_stages
        self.state_dict = state_dict  # if use, save computed state transitions for 
                                    # future reference
        self.n_trials = n_trials
        
    
    # helper functions
    @staticmethod
    def encode_state_key(lv, max_hp, cur_hp, att, def_):
        return f"{lv}/{max_hp}/{cur_hp}/{att}/{def_}"
    
    
    @staticmethod
    def decode_state_key(state_key):
        lv, max_hp, cur_hp, att, def_ = state_key.split('/')
        return int(lv), int(max_hp), int(cur_hp), int(att), int(def_)
      
        
    @staticmethod
    def save_state(state, action, info, state_trans):
        state_trans[(state, action)] = info
    

    def get_action_results(self, state, action, state_trans=None, win_rates=None):
        if state_trans is not None and (state, action) in state_trans:
            return state_trans[(state, action)]
        else:
            # next state has not been calculated before
            # simulate state action
            lv, max_hp, cur_hp, att, def_ = self.decode_state_key(state)
                
            if action == 1:
                next_state = self.encode_state_key(lv, max_hp, max_hp, att, def_)
                # save state action for future computation
                # probability of transition to the next state for action 1 is always 1
                # expected level for the next state will also be the same
                ns_info = (next_state, 1, lv)
                if state_trans is not None:
                    state_trans[(state, action)] = ns_info
                # return expected next state, transition probability and expected lv at next state
                return ns_info
            elif action == 2:
                if cur_hp == max_hp:
                    next_state = self.encode_state_key(lv, max_hp+30, cur_hp+30, att, def_)
                else:
                    next_state = self.encode_state_key(lv, max_hp+30, cur_hp, att, def_)
                # save state action for future computation
                # probability of transition to the next state for action 2 is always 1
                # expected level for the next state will also be the same
                ns_info = (next_state, 1, lv)
                if state_trans is not None:
                    state_trans[(state, action)] = ns_info
                # return next state info
                return ns_info
            elif action == 3:
                next_state = self.encode_state_key(lv, max_hp, cur_hp, att+3, def_)
                # save state action for future computation
                # probability of transition to the next state for action 3 is always 1
                # expected level for the next state will also be the same
                ns_info = (next_state, 1, lv)
                if state_trans is not None:
                    state_trans[(state, action)] = ns_info
                # return next state info
                return ns_info
            elif action == 4:
                next_state = self.encode_state_key(lv, max_hp, cur_hp, att, def_+3)
                # save state action for future computation
                # probability of transition to the next state for action 4 is always 1
                # expected level for the next state will also be the same
                ns_info = (next_state, 1, lv)
                if state_trans is not None:
                    state_trans[(state, action)] = ns_info
                # return expected next state info 
                return ns_info
            elif action == 5:
                # need to consider both cases of win and lose
                # simulation stochastically by setting up trials with the current stats
                hero = Hero("action_5")
                hero.any_stats(lv, max_hp, cur_hp, att, def_)
                win_rate, avg_hp = hero.trial(self.n_trials, "mini", win_rates)
                
                # if win
                exp_lv_win = lv+1
                trans_prob_win = win_rate
                next_state_win = self.encode_state_key(exp_lv_win, max_hp+10, avg_hp, att+1, def_+1)
                
                # if lose
                exp_lv_lose = lv
                next_state_lose = self.encode_state_key(exp_lv_lose, max_hp, int(0.5*max_hp), att, def_)
                trans_prob_lose = 1 - win_rate
                
                
                # save results to state dict
                ns_info = [(next_state_win, trans_prob_win, exp_lv_win), 
                           (next_state_lose, trans_prob_lose, exp_lv_lose),]
                
                # save state trans to manager dict (if requested)
                if state_trans is not None:
                    state_trans[(state, action)] = ns_info
                
                return ns_info
            
            elif action == 6:
                # need to consider both cases of win and lose
                # simulation stochastically by setting up trials with the current stats
                hero = Hero("action_6")
                hero.any_stats(lv, max_hp, cur_hp, att, def_)
                win_rate, avg_hp = hero.trial(self.n_trials, "champion", win_rates)
                
                # if win
                exp_lv_win = lv+2
                trans_prob_win = win_rate
                next_state_win = self.encode_state_key(exp_lv_win, max_hp+20, avg_hp, att+2, def_+2)
                
                # if lose
                exp_lv_lose = lv
                next_state_lose = self.encode_state_key(exp_lv_lose, max_hp, int(0.5*max_hp), att, def_)
                trans_prob_lose = 1 - win_rate
                
                
                # save results to state dict
                ns_info = [(next_state_win, trans_prob_win, exp_lv_win), 
                           (next_state_lose, trans_prob_lose, exp_lv_lose),]
                
                # save state trans to manager dict (if requested)
                if state_trans is not None:
                    state_trans[(state, action)] = ns_info
                
                return ns_info
            
            elif action == 7:
                # need to consider both cases of win and lose
                # simulation stochastically by setting up trials with the current stats
                hero = Hero("action_7")
                hero.any_stats(lv, max_hp, cur_hp, att, def_)
                win_rate, avg_hp = hero.trial(self.n_trials, "ultimate", win_rates)
                
                # if win
                exp_lv_win = lv+3
                trans_prob_win = win_rate
                next_state_win = self.encode_state_key(exp_lv_win, max_hp+30, avg_hp, att+3, def_+3)
                
                # if lose
                exp_lv_lose = lv
                next_state_lose = self.encode_state_key(exp_lv_lose, max_hp, int(0.5*max_hp), att, def_)
                trans_prob_lose = 1 - win_rate
                
                
                # save results to state dict
                ns_info = [(next_state_win, trans_prob_win, exp_lv_win), 
                           (next_state_lose, trans_prob_lose, exp_lv_lose),]
                
                # save state trans to manager dict (if requested)
                if state_trans is not None:
                    state_trans[(state, action)] = ns_info
                
                return ns_info
            

    def recursion(self, state, action, cur_depth, final_depth, state_trans=None, win_rates=None):
        # first simulate the next state from the current state after certain action
        ns_info = self.get_action_results(state, action, state_trans, win_rates)
            
        # base case: current stage is the final stage, reached end of recursion
        if cur_depth == final_depth:
            if action in range(1, 5): # action 1-4, expected max future level J_{k+1} is 1 * exp_lv
                next_state, trans_prob, exp_lv = ns_info
                return exp_lv, ["end"]
            else:  # action 5-7, J_{k+1} = p_win * exp_lv_win + (1-p) * exp_lv_lose
                next_state_win, trans_prob_win, exp_lv_win = ns_info[0]
                next_state_lose, trans_prob_lose, exp_lv_lose = ns_info[1]
                exp_lv = trans_prob_win * exp_lv_win + trans_prob_lose * exp_lv_lose
                return exp_lv, ["end"]
        else:
            max_exp_lv = -np.inf
            optimal_steps = None
                 
            if action in range(1, 5):
                next_state, trans_prob, exp_lv = ns_info
                
                # simulate actions for the next state
                for next_action in range(1, 8):
                    exp_lv, racts = self.recursion(next_state, next_action, cur_depth+1, final_depth, 
                                                  state_trans, win_rates)
                    if exp_lv > max_exp_lv:
                        max_exp_lv = exp_lv
                        optimal_steps = [next_action, racts]
                        
            else:
                next_state_win, trans_prob_win, exp_lv_win = ns_info[0]
                next_state_lose, trans_prob_lose, exp_lv_lose = ns_info[1]
                
                for next_action in range(1, 8):
                    # need to simulate for both win and lose situations
                    exp_lv_win, racts_win = self.recursion(next_state_win, next_action, 
                                                           cur_depth+1, final_depth,
                                                          state_trans, win_rates)
                    
                    exp_lv_lose, racts_lose = self.recursion(next_state_lose, next_action, 
                                                           cur_depth+1, final_depth,
                                                            state_trans, win_rates)
            
                    exp_lv = trans_prob_win * exp_lv_win + trans_prob_lose * exp_lv_lose
                
                    if exp_lv > max_exp_lv:
                        max_exp_lv = exp_lv
                        # add both win and lose cases to decisions
                        if len(racts_win) == 1:
                            racts = [("win:", racts_win), ("lose:", racts_lose)]
                        else:
                            racts = [racts_win[0], ("win:", racts_win[1]), ("lose:", racts_lose[1])]
          
                        optimal_steps = [next_action, racts]
                
            return max_exp_lv, optimal_steps
    
    
    # the optimal solution uses the same recursion structure as in "recursion"
    # but simulates state 0
    def optimal_solution(self, use_db_state_trans=True, use_db_win_rates=True):
        # get initial stats
        lv = self.hero.level
        max_hp = self.hero.max_vitality
        cur_hp = self.hero.current_vitality
        att = self.hero.attack_power
        def_ = self.hero.defense_power
        
        # encode the initial state
        state_0 = self.encode_state_key(lv, max_hp, cur_hp, att, def_)
        
        max_exp_lv = -np.inf
        best_state_0_action = None
        best_recursion_actions = None
    
        #! the following code requires multiprocessing
        def worker(action, manager_dict, state_trans=None, win_rates=None):
            exp_lv, racts = self.recursion(state_0, action, 1, self.n_stages,
                                           state_trans, win_rates,)
            # put result to multiprocessing manager dictionary
            manager_dict[action] = (exp_lv, racts)
            
        
        manager = multiprocessing.Manager()
        
        if use_db_state_trans:
            state_trans = manager.dict(self.state_dict)
        else:
            state_trans = None
            
        if use_db_win_rates:
            win_rates = manager.dict(db_win_rate.db)
        else:
            win_rates = None
            
        manager_dict = manager.dict()
        
        jobs = []
        
        print("Multiprocessing.. (this might take awhile)")
        
        # simulate actions for the next state
        for action in range(1, 8):
            p = multiprocessing.Process(target=worker, args=(action, manager_dict, 
                                                             state_trans, win_rates))
            jobs.append(p)
            p.start()
                    
        for proc in jobs: proc.join()

        for action in range(1, 8):
            exp_lv, racts = manager_dict[action]
            
            if exp_lv > max_exp_lv:
                max_exp_lv = exp_lv
                best_state_0_action = action
                best_recursion_actions = racts
        
        # update state dict and win rate db if such dicts are used
        if use_db_state_trans:
            self.state_dict.update(state_trans)
        if use_db_win_rates:
            db_win_rate.db.update(win_rates)
            
            
        return round(max_exp_lv, 2), [best_state_0_action, best_recursion_actions]
    
state_dict = dict()  # DO NOT DELETE THIS INSTANCE! MIGHT LOSE ALL COMPUTED STATE TRANSITIONS

explore the optimal solution

In [26]:
print("current number of saved state transition entries: ", len(state_dict))
print("current number of saved win rate entries: ", len(db_win_rate.db))

current number of saved state transition entries:  0
current number of saved win rate entries:  195


In [64]:
# the state place for the optimal solution is ginormous!
# we can look up up to 7 steps (O(7^7)) in a relatively short amount of time 
n_stages = 7
hero = Hero("optimal")
solution = Forward_recursion(hero, n_stages, state_dict, n_trials=100)
exp_lv_op, steps_op = solution.optimal_solution(use_db_state_trans=True, use_db_win_rates=True)

print(f"Optimal solution, instance {n_stages}")
print(f"Expected level achieved: {exp_lv_op}")
print(f"Optimal steps: {steps_op}")

Multiprocessing.. (this might take awhile)
Optimal solution, instance 7
Expected level achieved: 2.72
Optimal steps: [2, [2, [2, [2, [5, [1, [5, ('win:', ['end']), ('lose:', ['end'])]]]]]]]


In [67]:
game = Game(hero)
game.play(n_stages, )

*********************************************
Stage 1: *optimal* Level 1, HP: 100
MAX_HP: 100, ATT: 20, DEF: 10
*********************************************
Choose an action:
1: Restore Full Health
2: +30 Max HP
3: Attack +3
4: Defense + 3
5: Fight a mini boss (win rate: 2.5%)
6: Fight a champion (win rate: 0.0%)
7: Fight an ultimate boss (win rate: 0.0%)
8: Give up.. 
2
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
*optimal* max vitality + 30
*********************************************
Stage 2: *optimal* Level 1, HP: 130
MAX_HP: 130, ATT: 20, DEF: 10
*********************************************
Choose an action:
1: Restore Full Health
2: +30 Max HP
3: Attack +3
4: Defense + 3
5: Fight a mini boss (win rate: 15.5%)
6: Fight a champion (win rate: 0.0%)
7: Fight an ultimate boss (win rate: 0.0%)
8: Give up.. 
2
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
*optimal* max vitality + 30
*********************************************
Stage 3: *optimal* Level 1, HP: 160
MAX_HP: 160, ATT: 2

In [65]:
# transition table after forward recursion run
print("current number of saved state transition entries: ", len(state_dict))
print("current number of saved win rate entries: ", len(db_win_rate.db))

current number of saved state transition entries:  11767
current number of saved win rate entries:  2754


Here we visualize the saved state transitions

            "current state"             "next state"
FORMAT: ( 'lv/max_hp/cur_hp/att/def', action) = ( 'lv/max_hp/cur_hp/att/def', trans_prob, exp_lv)

In [31]:
# display 10 samples from the state dict
# ref: stackoverflow.com/questions/7971618/python-return-first-n-keyvalue-pairs-from-dict
from itertools import islice

def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))

n = 10
for key, value in take(n, state_dict.items()):
    print("State, action: ", key, "Next state, trans_prob, level: ", value)

State, action:  ('1/100/100/20/10', 1) Next state, trans_prob, level:  ('1/100/100/20/10', 1, 1)
State, action:  ('1/100/100/20/10', 2) Next state, trans_prob, level:  ('1/130/130/20/10', 1, 1)
State, action:  ('1/100/100/20/10', 3) Next state, trans_prob, level:  ('1/100/100/23/10', 1, 1)
State, action:  ('1/100/100/20/10', 4) Next state, trans_prob, level:  ('1/100/100/20/13', 1, 1)
State, action:  ('1/100/100/20/10', 5) Next state, trans_prob, level:  [('2/110/10/21/11', 0.031, 2), ('1/100/50/20/10', 0.969, 1)]
State, action:  ('1/100/100/20/10', 6) Next state, trans_prob, level:  [('3/120/1/22/12', 0.0, 3), ('1/100/50/20/10', 1.0, 1)]
State, action:  ('1/100/100/20/10', 7) Next state, trans_prob, level:  [('4/130/1/23/13', 0.0, 4), ('1/100/50/20/10', 1.0, 1)]
State, action:  ('1/130/130/20/10', 1) Next state, trans_prob, level:  ('1/130/130/20/10', 1, 1)
State, action:  ('1/130/130/20/10', 2) Next state, trans_prob, level:  ('1/160/160/20/10', 1, 1)
State, action:  ('1/130/130/20/1

Approximate Dynamic Programming 
=============================
In the final section, we will implement the

parametric approximation algorithm via fitted value iteration

The structure of the code is like a hybrid of rollout and forward recursion

idea: use monte carlo simulation and value iteration 

to approximate the maximum expected lv increase \hat{r}_k for state k 

from the cost of the current state and the maximum exp lv increase of J_{k-1}

**\beta_k^s = max E(cost(x_k, u) + J_{k+1, r_k+1})**

**\hat{r}_k = arg min (\sum J_k + \beta_k^s)^2**  (solve by gradient)

In [8]:
class ADP:
    def __init__(self, n_stages, n_trials=100, q=100, 
                 use_state_db=True, use_win_rate_db=True):
        self.n_stages = n_stages
        self.n_trials = n_trials
        self.q = q
    
        # use transition table
        self.state_dict = state_dict if use_state_db else None

        # use win rate data
        self.win_rate_db = db_win_rate.db if use_win_rate_db else None
    
        self.hero = Hero("ADP")
        # instantiate a recursion instance for calculating state transitions
        self.recursion = Forward_recursion(self.hero, n_stages, 
                                           self.state_dict, n_trials)
        
        # initializes a table for the value functions per iteration
        # list [cur_hp/att/def]
        self.value_dict = dict.fromkeys(range(self.n_stages+1), 0.0)
        
        # initialize the max. exp. reward (\tilde(J)_{k}, k = 0..N) table
        self.exp_reward_db = {}
    
        # sample from state 0 (initial hero stats)
        # get initial stats
        lv = self.hero.level
        max_hp = self.hero.max_vitality
        cur_hp = self.hero.current_vitality
        att = self.hero.attack_power
        def_ = self.hero.defense_power

        # encode the initial state
        self.state_0 = self.recursion.encode_state_key(lv, max_hp, cur_hp, att, def_)
        
    
    # apply a linear mapping to player stats to approximate 
    # a value state for value iteration
    # the state is non inversible
    # linear mapping formula: (win_rate_mini + win_rate_champion + win_rate_ult) * 100
    def encode_state(self, state):
        lv, max_hp, cur_hp, att, def_ = self.recursion.decode_state_key(state)
        self.hero.any_stats(lv, max_hp, cur_hp, att, def_)
        mini_boss_win_rate, champion_win_rate, ult_boss_win_rate = self.hero.get_win_rates(self.n_trials, 
                                                                                          self.win_rate_db)
            
        return round(100 * (mini_boss_win_rate + champion_win_rate + ult_boss_win_rate))

    
    # function: sample up to q states for every stage k 
    # use the forward recursion module to use the get action result function
    def sample_states(self):
        
        # initialize a table to store all sampled states for stage 0 to N
        self.sampled_states = dict.fromkeys(range(self.n_stages+1), [])
        
        # tuple: (x_k, cost(exp. lv. increase))
        self.sampled_states[0] = [self.state_0 for _ in range(self.q)]
    
        for stage in range(1, self.n_stages+1):  # stage 1.. N 
            print(f"Sampling states. Stage: {stage}", end="\r")
            n_sampled = 0
            states = []
            while n_sampled < self.q:
                # randomly select a previous state
                previous_state = np.random.choice(self.sampled_states[stage-1])

                # randomly select an action between 1-7
                action = np.random.randint(low=1, high=8)

                # sample a random state x_k
                if action in range(1,5):  # action 1 - 4
                    new_state, trans_prob, exp_lv, = self.recursion.get_action_results(previous_state, action,
                                                                          self.state_dict, self.win_rate_db)
                    
                else:
                    new_states = self.recursion.get_action_results(previous_state, action,
                                                      self.state_dict, self.win_rate_db)

                    new_state_win, trans_prob_win, exp_lv_win = new_states[0]
                    new_state_lose, trans_prob_lose, exp_lv_lose = new_states[1]

                    # choose between the win and lose case to add to samples
                    # according to their probabilities
                    new_state = np.random.choice([new_state_win, new_state_lose], 
                                                 p=[trans_prob_win, trans_prob_lose])

                    
                states.append(new_state)
                n_sampled += 1

            self.sampled_states[stage] = states
                

                
    # recursive function to trace the "exact" future reward (not approximated)
    # given state, step i
    # returns: exp reward J_k(x_k, r_k)
    # input state: a non encoded state representation: lv/maxhp/curhp/att/def_
    def find_exp_reward(self, state, stage):
        assert self.exp_reward_db is not None, "No exp. reward table found."
        
        # encode the state from input
        encode_state = self.encode_state(state)
    
        # all states at the stage N have expected return reward of 0
        if stage == self.n_stages:
            if (stage, encode_state) not in self.exp_reward_db:
                self.exp_reward_db[(stage, encode_state)] = 0.0 
            return 0.0
        
        # check if the state exp reward is already in the table
        if (stage, encode_state) in self.exp_reward_db:
            # J_{k+1}(x_k)
            return self.exp_reward_db[(stage, encode_state)]
        
        # the exp reward for the state cannot be found
        # from now on, approximate the exp reward
        
        future_rewards = []
        for action in range(1, 8):
            ns_info = self.recursion.get_action_results(state, action, 
                                                        self.state_dict, self.win_rate_db)
            
            if action in range(1, 5):
                next_state, _, _, = ns_info   # next state is NOT encoded
                
                action_reward = self.find_exp_reward(next_state, stage+1) * self.value_dict[stage+1]
                
            else:  # action 5 - 7
                next_state_win, prob_win, _, = ns_info[0]
                next_state_lose, prob_lose, _, = ns_info[1]
                
                win_reward = self.find_exp_reward(next_state_win, stage+1)
                
                lose_reward = self.find_exp_reward(next_state_lose, stage+1)
                
                immediate_reward = prob_win * (action - 4)  # action 5: reward 1 (lv+)
                                                         # action 6: reward 2 (lv+)
                                                         # action 7: reward 3 (lv+)
                
                future_reward = (prob_win * win_reward + prob_lose * lose_reward)
                
                action_reward = immediate_reward + future_reward * self.value_dict[stage+1]
                
            future_rewards.append(action_reward)
            
        max_exp_reward = max(future_rewards)
        
        # save to exp reward table
        self.exp_reward_db[(stage, encode_state)] = max_exp_reward
        
        return max_exp_reward
    
    
    
    def value_iteration(self):
        
        assert self.sampled_states is not None, "No sampled states are found."
        assert self.exp_reward_db is not None, "No exp. reward table found."
        
        k = self.n_stages - 1
        # start value iteration
        
        while k >= 0:

            # use pre sampled states for step k
            states_k = self.sampled_states[k]

            # use the value function r_{k+1} from the previous step (value iteration!)
            #value_k = self.value_dict[k+1]

            # table for saving (x^s_k, beta^s_k) 
            state_reward_k = dict()

            for counter, state in enumerate(states_k, 1):
                
                print(f"Evaluating states.. Stage: {k+1}, q: {counter}", end="\r")
                max_beta = self.find_exp_reward(state, k)

                # update {state_reward}
                state_reward_k[state] = max_beta

            # update value function using gradient of MSE
            value_k = 0.0
            for state, beta in state_reward_k.items():
                # encode the state from input
                encode_state = self.encode_state(state)
                J = self.exp_reward_db[(k, encode_state)]
                value_k += beta / J if (J != 0) else 0.0
            value_k = value_k / self.q
            
            self.value_dict[k] = value_k

            k -= 1
            
            
    def solution(self):
        
        # start from state 0
        last_state = self.state_0
        
        decisions = [last_state]
        
        for stage in range(self.n_stages):
            best_action = None
            best_state = None
            max_exp_return = -np.inf
            
            for action in range(1, 8):
                ns_info = self.recursion.get_action_results(last_state, action,
                                                           self.state_dict, self.win_rate_db)
                if action in range(1,5):
                    next_state, trans_prob, _ = ns_info
                    
                    # use the calculated value function
                    exp_return = self.find_exp_reward(next_state, stage+1)
                    

                else:
                    next_state_win, prob_win, _ = ns_info[0]
                    next_state_lose, prob_lose, _ = ns_info[1]
                    
                    win_reward = self.find_exp_reward(next_state_win, stage+1)
                
                    lose_reward = self.find_exp_reward(next_state_lose, stage+1)

                    immediate_reward = prob_win * (action - 4)  # action 5: reward 1 (lv+)
                                                             # action 6: reward 2 (lv+)
                                                             # action 7: reward 3 (lv+)

                    future_reward = (prob_win * win_reward + prob_lose * lose_reward)

                    exp_return = immediate_reward + future_reward * self.value_dict[stage]
                    
                    # randomly transit to either a state in respects to prob_win and prob_lose
                    next_state = np.random.choice([next_state_win, next_state_lose], p=[prob_win, prob_lose])

                if exp_return > max_exp_return:
                    max_exp_return = exp_return
                    best_action = action
                    best_state = next_state
                        
            print(f"Stage {stage+1}, best action: {best_action}")
            
            last_state = best_state
            decisions.append(last_state)
            
        return decisions

Try out value iteration
=================

Small instance of N=10, q=100 sample per stage

In [ ]:
parametric = ADP(n_stages=10, n_trials=100, q=1000, 
                 use_state_db=True, use_win_rate_db=True)
parametric.sample_states()
parametric.value_iteration()

sample of the value function reward table

In [11]:
n=0
for key, value in parametric.exp_reward_db.items():
    if n > 100:
        break
    
    print(key, value)
    n += 1

(10, 52) 0.0
(10, 1) 0.0
(10, 2) 0.0
(10, 0) 0.0
(9, 1) 0.01
(10, 97) 0.0
(10, 18) 0.0
(10, 30) 0.0
(10, 28) 0.0
(9, 18) 0.18
(10, 88) 0.0
(10, 13) 0.0
(10, 25) 0.0
(10, 21) 0.0
(9, 13) 0.13
(10, 86) 0.0
(10, 5) 0.0
(10, 17) 0.0
(10, 8) 0.0
(9, 5) 0.05
(10, 74) 0.0
(10, 90) 0.0
(10, 78) 0.0
(9, 74) 0.74
(10, 76) 0.0
(9, 2) 0.02
(10, 99) 0.0
(10, 96) 0.0
(10, 10) 0.0
(9, 88) 0.88
(9, 0) 0.0
(10, 58) 0.0
(10, 80) 0.0
(10, 70) 0.0
(9, 58) 0.58
(10, 100) 0.0
(10, 33) 0.0
(10, 43) 0.0
(9, 33) 0.33
(10, 7) 0.0
(10, 16) 0.0
(9, 7) 0.07
(10, 31) 0.0
(9, 17) 0.17
(10, 35) 0.0
(10, 41) 0.0
(9, 25) 0.25
(10, 72) 0.0
(10, 89) 0.0
(10, 9) 0.0
(9, 72) 0.72
(9, 96) 0.96
(10, 22) 0.0
(10, 49) 0.0
(10, 34) 0.0
(9, 22) 0.22
(10, 61) 0.0
(10, 82) 0.0
(10, 77) 0.0
(9, 61) 0.61
(10, 4) 0.0
(9, 4) 0.04
(10, 85) 0.0
(10, 66) 0.0
(9, 52) 0.52
(10, 53) 0.0
(10, 73) 0.0
(9, 53) 0.53
(10, 91) 0.0
(9, 86) 0.86
(10, 95) 0.0
(9, 10) 0.1
(10, 50) 0.0
(9, 100) 1.0
(10, 93) 0.0
(10, 98) 0.0
(9, 93) 0.93
(10, 62) 0.0
(

Solution via ADP

In [12]:
decisions = parametric.solution()

Stage 1, best action: 2
Stage 2, best action: 2
Stage 3, best action: 3
Stage 4, best action: 2
Stage 5, best action: 2
Stage 6, best action: 3
Stage 7, best action: 2
Stage 8, best action: 5
Stage 9, best action: 1
Stage 10, best action: 5


what about 1000 samples per stage? N=10, q=1000

In [16]:
parametric = ADP(n_stages=10, n_trials=100, q=1000, 
                 use_state_db=True, use_win_rate_db=True)
parametric.sample_states()
parametric.initialize_exp_reward_table()
parametric.value_iteration()

In [17]:
decisions = parametric.solution()

Stage 1, best action: 2
Stage 2, best action: 2
Stage 3, best action: 2
Stage 4, best action: 5
Stage 5, best action: 1
Stage 6, best action: 5
Stage 7, best action: 1
Stage 8, best action: 5
Stage 9, best action: 1
Stage 10, best action: 5


Instance 2: N=25, q=100

In [18]:
parametric = ADP(n_stages=25, n_trials=100, q=100, 
                 use_state_db=True, use_win_rate_db=True)
parametric.sample_states()
parametric.initialize_exp_reward_table()
parametric.value_iteration()

In [19]:
decisions = parametric.solution()

Stage 1, best action: 2
Stage 2, best action: 2
Stage 3, best action: 2
Stage 4, best action: 2
Stage 5, best action: 2
Stage 6, best action: 3
Stage 7, best action: 4
Stage 8, best action: 4
Stage 9, best action: 5
Stage 10, best action: 5
Stage 11, best action: 1
Stage 12, best action: 5
Stage 13, best action: 5
Stage 14, best action: 1
Stage 15, best action: 5
Stage 16, best action: 5
Stage 17, best action: 5
Stage 18, best action: 1
Stage 19, best action: 5
Stage 20, best action: 5
Stage 21, best action: 5
Stage 22, best action: 5
Stage 23, best action: 1
Stage 24, best action: 6
Stage 25, best action: 5


**Playground**
===============

Try out the game yourself below \
Enter your hero name and number of total stages \
When prompted, enter the preferred action at each stage! (please enter the numbers displayed **ONLY**)

In [24]:
run_game() 

Enter hero name: 
WW
Enter the number of days to train and level up: (DIGITS ONLY)
10
Starting game...
*********************************************
Stage 1: *WW* Level 1, HP: 100
MAX_HP: 100, ATT: 20, DEF: 10
*********************************************
Choose an action:
1: Restore Full Health
2: +30 Max HP
3: Attack +3
4: Defense + 3
5: Fight a mini boss (win rate: 3.5%)
6: Fight a champion (win rate: 0.0%)
7: Fight an ultimate boss (win rate: 0.0%)
8: Give up.. 
2
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
*WW* max vitality + 30
*********************************************
Stage 2: *WW* Level 1, HP: 130
MAX_HP: 130, ATT: 20, DEF: 10
*********************************************
Choose an action:
1: Restore Full Health
2: +30 Max HP
3: Attack +3
4: Defense + 3
5: Fight a mini boss (win rate: 13.0%)
6: Fight a champion (win rate: 0.0%)
7: Fight an ultimate boss (win rate: 0.0%)
8: Give up.. 
2
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
*WW* max vitality + 30
******************

5
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Erika, boss type: mini boss
Upper hand: boss
### Round 2 ###
WW Health: 240, Erika Health: 150
Erika does amage: 11
WW does damage: 7.
### Round 3 ###
WW Health: 229, Erika Health: 143
Erika does amage: 10
WW does damage: 6.
### Round 4 ###
WW Health: 219, Erika Health: 137
Erika does amage: 11
WW does damage: 9.
### Round 5 ###
WW Health: 208, Erika Health: 128
Erika does amage: 11
WW does damage: 7.
### Round 6 ###
WW Health: 197, Erika Health: 121
Erika does amage: 13
WW does damage: 7.
### Round 7 ###
WW Health: 184, Erika Health: 114
Erika does amage: 10
WW does damage: 7.
### Round 8 ###
WW Health: 174, Erika Health: 107
Erika does amage: 9
WW does damage: 8.
### Round 9 ###
WW Health: 165, Erika Health: 99
Erika does amage: 13
WW does damage: 7.
### Round 10 ###
WW Health: 152, Erika Health: 92
Erika does amage: 11
WW does damage: 7.
### Round 11 ###
WW Health: 141, Erika Health: 85
Erika does amage: 10
WW does damage: 7.
### Round

#THE END#